<a href="https://colab.research.google.com/github/manojach87/miscellaneous/blob/master/coingecko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.coingecko.com/api/documentations/v3#/operations/ping

In [1]:
def readJsonLink(link):
  import urllib.request, json 
  with urllib.request.urlopen(link) as url:
      data = json.loads(url.read().decode())
      return data

In [2]:
def getMarkets(url="https://api.coingecko.com/api/v3/coins/markets?vs_currency=USD&order=market_cap_desc&per_page=250&sparkline=false",pages=10):
  marketsData=list()
  for i in range(1,pages):
    #print(i)
    marketsData.append(readJsonLink(url+"&page="+str(i)))
  return marketsData
#marketsData

In [3]:
def createDf(url,pages=10):
  import pandas as pd
  marketsData=getMarkets(url,pages=pages+1)
  marketsDf=pd.DataFrame()
  for i in range(0,len(marketsData)):
    #print(i)
    marketsDf=marketsDf.append(pd.DataFrame(marketsData[i]))
  marketsDf=marketsDf.reset_index()
  return marketsDf

In [4]:
marketsDf=createDf(url="https://api.coingecko.com/api/v3/coins/markets?vs_currency=USD&order=market_cap_desc&per_page=250&sparkline=false",pages=28)

In [5]:
#marketsDf[marketsDf.index==0]
marketsDf

,index,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,low_24h,price_change_24h,price_change_percentage_24h,market_cap_change_24h,market_cap_change_percentage_24h,circulating_supply,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,0,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,63717.00,1.182688e+12,1,1.329427e+12,7.500317e+10,63693.00,60239.00,3076.380000,5.07314,4.871448e+10,4.29591,1.868207e+07,2.100000e+07,21000000.0,63693.00,-0.70193,2021-04-14T00:09:07.264Z,67.810000,9.317082e+04,2013-07-06T00:00:00.000Z,None,2021-04-14T03:35:11.546Z
1,1,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/...,2342.86,2.696550e+11,2,NaN,3.790469e+10,2340.19,2156.20,179.080000,8.27639,1.959643e+10,7.83674,1.154604e+08,NaN,NaN,2315.84,0.00000,2021-04-14T03:17:13.460Z,0.432979,5.347630e+05,2015-10-20T00:00:00.000Z,"{'times': 48.41497708379405, 'currency': 'btc'...",2021-04-14T03:36:34.807Z
2,2,binancecoin,bnb,Binance Coin,https://assets.coingecko.com/coins/images/825/...,562.42,8.664144e+10,3,9.561206e+10,8.893056e+09,580.82,536.97,-11.478109,-2.00003,-1.276963e+09,-1.45244,1.545337e+08,1.705337e+08,170533651.9,610.06,-8.31646,2021-04-12T09:32:32.635Z,0.039818,1.404603e+06,2017-10-19T00:00:00.000Z,None,2021-04-14T03:35:13.099Z
3,3,ripple,xrp,XRP,https://assets.coingecko.com/coins/images/44/l...,1.79,8.170350e+10,4,NaN,2.850095e+10,1.88,1.44,0.350404,24.40893,1.585778e+10,24.08324,4.597332e+10,1.000000e+11,NaN,3.40,-47.54491,2018-01-07T00:00:00.000Z,0.002686,6.626347e+04,2014-05-22T00:00:00.000Z,None,2021-04-14T03:35:39.515Z
4,4,cardano,ada,Cardano,https://assets.coingecko.com/coins/images/975/...,1.46,4.675352e+10,5,NaN,6.338972e+09,1.47,1.29,0.158310,12.15491,5.094471e+09,12.22896,3.203216e+10,4.500000e+10,NaN,1.48,-0.87665,2021-03-18T15:55:08.465Z,0.019253,7.506175e+03,2020-03-13T02:22:55.044Z,None,2021-04-14T03:37:02.026Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6681,181,dain,dnc,Dain,https://assets.coingecko.com/coins/images/1317...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+09,NaN,NaN,0.00000,None,NaN,0.000000e+00,None,None,None
6682,182,bidesk,bdk,Bidesk,https://assets.coingecko.com/coins/images/1288...,NaN,NaN,None,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.997720e+07,NaN,NaN,0.00000,None,NaN,0.000000e+00,None,None,2021-04-14T03:30:55.750Z
6683,183,fridaybeers,beer,FridayBeers,https://assets.coingecko.com/coins/images/1285...,NaN,NaN,None,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.200000e+04,32000.0,NaN,0.00000,None,NaN,0.000000e+00,None,None,2021-03-17T22:33:30.919Z
6684,184,thinkium,tkm,Thinkium,https://assets.coingecko.com/coins/images/1171...,NaN,NaN,None,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,None,NaN,0.000000e+00,None,None,2021-04-14T03:23:26.085Z


In [6]:
len(marketsDf)

6686

In [7]:
btcSupply=marketsDf[marketsDf.symbol=="btc"].circulating_supply[0]
btcSupply

18682068.0

In [8]:
marketsDf["btc_relative_val"]=marketsDf.circulating_supply/btcSupply*marketsDf.current_price

In [9]:
marketsDf.sort_values(by=["btc_relative_val"], ascending=[0], inplace=True)

In [10]:
marketsDf[["symbol","btc_relative_val","current_price", "circulating_supply","market_cap"]][(marketsDf["btc_relative_val"]<20) & (marketsDf["btc_relative_val"]>10)]

,symbol,btc_relative_val,current_price,circulating_supply,market_cap
181,sfp,19.859240,3.430000,1.081667e+08,367124955.0
180,paid,19.651470,4.660000,7.878328e+07,367646469.0
182,woo,19.429467,0.875336,4.146781e+08,363753976.0
183,badger,19.366919,45.420000,7.965964e+06,359581165.0
185,bts,19.282041,0.132916,2.710196e+09,358268374.0
...,...,...,...,...,...
274,xcm,10.437659,0.643154,3.031888e+08,194997029.0
275,lto,10.238835,0.681934,2.805002e+08,190779240.0
276,mask,10.117666,17.320000,1.091333e+07,188961936.0
277,zmt,10.034137,4.670000,4.014099e+07,187495982.0


In [11]:
mmDf=createDf(url="https://api.coingecko.com/api/v3/exchanges/binance/tickers?",pages=10)

In [41]:
import pandas as pd
binanceDf=pd.DataFrame([ row for row in mmDf.tickers])[["coin_id","last"]]


In [46]:
import plotly.express as px
fig = px.bar(marketsDf.head(200), x='symbol', y='btc_relative_val')
fig.show()